## Import cell

In [ ]:
#%%Import modules and import CSV data
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, shape
import os
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt

#make directory/pathing more intuitive later
user_directory = os.fspath(Path.home()) #get home directory and convert to path string
file_directory = "\Documents\GitHub\BikewaySim\TransportSim" #directory of bikewaysim network processing code

#change this to where you stored this folder
os.chdir(user_directory+file_directory)

## File paths for paths

In [ ]:
bikewaysim_fp_nodes = 'bikewaysim_network/2020 nodes with latlon/2020_nodes_latlon.geojson'
bikewaysim_fp_links = 'bikewaysim_network/2020 links/2020_links.geojson'
bikewaysim_date_name = '062221_072540' #for bikewaysim
bikewaysim_pref_date_name = '062421_092503' #for bikewaysim but with preferences, instead of distances

abm_fp_nodes = 'abm/2020 nodes with latlon/2020_nodes_latlon.geojson'
abm_fp_links = 'abm/2020 links/2020_links.geojson'
abm_date_name = '062121_200336' # for abm


## Read in files

In [ ]:
def map_trips(fp_nodes, fp_links, date_name):
    #NOTE need to change this filepath everytime, integrate this into the existing bikewaysim code
    fp_results_csv = fr'trips_bws\results\{date_name}\paths_bike.csv'

    #read in CSV files and set index to N and A
    df_node_csv = gpd.read_file(fp_nodes, ignore_geometry = True)
    df_links = gpd.read_file(fp_links)

    df_results_csv = pd.read_csv(fp_results_csv)

    #not implemented yet
    #read in OD file to get coordinates for OD pair
    #df_samples_in_csv = pd.read_csv(r'trips_bws/samples_in/od_pairs.csv')


    # Join data frames using N in the node csv to the A comlumn in the result csv then plot
    #drop
    df_node_csv = df_node_csv.drop_duplicates()
    
    #joined_results = df_results_csv.join(df_node_csv)
    joined_results = pd.merge(df_results_csv,df_node_csv, left_on='A', right_on='N', how='inner')
    
    #take out trips with less than two points
    test = joined_results.groupby(['trip_id'])['trip_id'].count()
    joined_results = pd.merge(joined_results, test, left_on = 'trip_id', right_index=True, how = 'left', suffixes=(None,'_count'))
    joined_results = joined_results[joined_results['trip_id_count'] > 1 ]
    
    
    joined_results = joined_results.sort_values(by=['trip_id','sequence'], axis=0, ascending = True)


    # Zip the coordinates into a point object and convert to a GeoDataFrame
    geometry = [Point(xy) for xy in zip(joined_results.X, joined_results.Y)]
    gdf = gpd.GeoDataFrame(joined_results, geometry=geometry)
    
    # Aggregate these points with the GroupBy
    gdf = joined_results.groupby(['trip_id'])['geometry'].apply(lambda x: LineString(x.to_list()))
    gdf = gpd.GeoDataFrame(gdf, geometry='geometry').set_crs(epsg=2240)
    
    #export to geojson
    gdf.to_file(rf"Analysis/{date_name}_trips.geojson", driver = 'GeoJSON')

    # Find num of trips on links
    trips_on_links = df_results_csv[['A','B']]
    trips_on_links = trips_on_links[-trips_on_links.isin(['origin','destination'])]
    trips_on_links['trips'] = 1
    trips_on_links['A_B'] = trips_on_links['A'] + "_" + trips_on_links['B']
    trips_on_links = trips_on_links.groupby(['A_B'])['trips'].sum()
    
    df_links_w_trips = pd.merge(df_links,trips_on_links, left_on = 'A_B', right_on = 'A_B' )
    df_links_w_trips.to_file(rf"Analysis/{date_name}_trips_on_links.geojson", driver = 'GeoJSON')
    
    return gdf, df_links_w_trips